# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from time import time

filename_features = "UCI HAR Dataset/features.txt"
filename_labels = "UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "UCI HAR Dataset/train/y_train.txt"

filename_subtest = "UCI HAR Dataset/test/subject_test.txt"
filename_xtest = "UCI HAR Dataset/test/X_test.txt"
filename_ytest = "UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")['nome_var']

labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])['subject_id']
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])['cod_label']

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])['subject_id']
X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])['cod_label']

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [3]:
%%time

clf = DecisionTreeClassifier(random_state=1234, ccp_alpha = 0.001).fit(X_train, y_train)

train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {train_score*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {test_score*100:.1f}')

Acurácia da melhor árvore na base de treino:    97.7
Acurácia da melhor árvore na base de teste:     93.8
CPU times: total: 5.89 s
Wall time: 6.79 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [4]:
%%time
prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

n = 1

colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
pc_test  = pd.DataFrame( pc_teste[:,:n], columns = colunas)

clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001)
clf = clf.fit(pc_train, y_train)
train_score = clf.score(pc_train, y_train)
test_score = clf.score(pc_test,y_test)

print(f'Acurácia da melhor árvore na base de treino:    {train_score*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {test_score*100:.1f}')

Acurácia da melhor árvore na base de treino:    49.7
Acurácia da melhor árvore na base de teste:     50.4
CPU times: total: 875 ms
Wall time: 439 ms


### Avalie a acurácia nas bases de treinamento e teste ee o tempo de processament
**R:** A acurácia foi relativamente baixa em comparação com a árvore de decisão original, o que se explica pelo uso de apenas uma variável para análise. Por outro lado, o tempo de processamento foi muito menor, visto que a árvore de decisão original consumiu cerca de 5,84 segundos, enquanto a árvore com PCA não consumiu nem 1 segundo.o

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [5]:
componentes = [1, 2, 5, 10, 50]

train_accuracies = []
test_accuracies = []
tempos = []

for n in componentes:
    prcomp = PCA(n_components=n).fit(X_train)
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)
    
    clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001)
    
    # Medindo o tempo de treinamento
    start_train = time()
    clf.fit(pc_treino, y_train)
    end_train = time()
    treino_tempo = end_train - start_train
    
    train_pred = clf.predict(pc_treino)
    train_acc = accuracy_score(y_train, train_pred)
    
    # Medindo o tempo de teste
    start_test = time()
    test_pred = clf.predict(pc_teste)
    end_test = time()
    teste_tempo = end_train - start_train
    test_acc = accuracy_score(y_test, test_pred)
    
    # Somando tempo de treinamento e teste
    tempo_total = treino_tempo + teste_tempo
    
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    tempos.append(tempo_total)
    
    print(f"Número de componentes: {n}")
    print(f"Acurácia no treino: {train_acc*100:.1f}")
    print(f"Acurácia no teste: {test_acc*100:.1f}")
    print(f"Tempo: {tempo_total:.2f}s")
    print('\n')

Número de componentes: 1
Acurácia no treino: 49.7
Acurácia no teste: 50.4
Tempo: 0.09s


Número de componentes: 2
Acurácia no treino: 62.1
Acurácia no teste: 59.1
Tempo: 0.09s


Número de componentes: 5
Acurácia no treino: 86.1
Acurácia no teste: 81.2
Tempo: 0.15s


Número de componentes: 10
Acurácia no treino: 89.9
Acurácia no teste: 85.2
Tempo: 0.25s


Número de componentes: 50
Acurácia no treino: 92.4
Acurácia no teste: 85.7
Tempo: 1.49s




## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

**R:** Podemos notar que, conforme aumentamos o número de componentes (variáveis) analisadas na árvore utilizando PCA, o tempo de processamento e a acurácia aumentam. Na última opção, com 50 variáveis, temos uma acurácia de cerca de 92.4% na base de treino e 85.7% na base de teste, em comparação com a árvore sem PCA, que teve 97,7% na base de treino e 93,8% na base de teste. No entanto, também é importante considerar que, utilizando as 50 variáveis, o tempo de processamento foi de 1.49s segundos, comparado aos 5,89 segundos sem o uso do PCA.